In [1]:
from lxml import etree
import sys
import json

In [51]:
# file_name=sys.argv[1]
# output_json=sys.argv[2]
# file_name=input()
f=open("./wk4-fsimage564.xml")
tree=etree.parse(f)
output_dict={}

In [3]:
def printf(elems):
    if (isinstance(elems, list)):
        for elem in elems:
            if isinstance(elem, str):
                print(elem) 
            else:
                print(etree.tostring(elem).decode('utf-8')) 
    else: # just a single element
        print(etree.tostring(elems).decode('utf-8'))

In [52]:
#find number of files
f_count=0
for element in tree.xpath("/fsimage/INodeSection/inode[type='FILE']"):
    f_count +=1
    #printf(element)
    #file =str(etree.tostring(element))
    #print(files)
    
output_dict["number of files"]=f_count
print(output_dict)

{'number of files': 29}


In [53]:
#find number of directories
d_count=0
for element in tree.xpath("/fsimage/INodeSection/inode[type='DIRECTORY']"):
    d_count +=1
    #printf(element)
    #file =str(etree.tostring(element))
    #print(files)
    
output_dict["number of directories"]=d_count
print(output_dict)

{'number of files': 29, 'number of directories': 9}


In [60]:
class Node(object):
    def __init__(self,name):
        self.name=name
        self.children=[]
    def addChild(self,name):
        self.children.append(Node(name))
        return self
    def addChildrenToParent(self,parent_inum,child_inum):
        queue=[self]
        while len(queue)>0:
            current=queue.pop(0)
            if parent_inum==current.name:
                current.addChild(child_inum)
            else:
                queue.extend(current.children)
    def deepFirstSearch(self,array=[]):
        array.append(self.name)
        for child in self.children:
            child.deepFirstSearch(array)
        return array
    def breadthFirstSearch(self,array=[]):
        queue=[self]
        while len(queue)>0:
            current=queue.pop(0)
            array.append(current.name)
            queue.extend(current.children)
        return array

    def maxDepth(self) -> int:
        d = 1
        for child in self.children:
            d = max(d, child.maxDepth() + 1)
        return d
    
    def countMaxDepth(self,depth=1,maxd=1):
        if len(self.children) !=0:
            depth +=1
            maxd=max(depth,maxd)
        for child in self.children:
            child.countMaxDepth(depth,maxdepth)
        return maxdepth

In [63]:

#for element in tree.xpath("/fsimage/INodeSection/inode[name[not(node())]]"):
 #   print(type(element.xpath("./id/text()")[0]))

root_node=tree.xpath("/fsimage/INodeSection/inode[name[not(node())]]")
root_num=root_node[0].xpath("./id/text()")[0]
root_num=int(root_num)
print(root_num)


dir_tree=Node(root_num)

for element in tree.xpath("/fsimage/INodeDirectorySection/directory"):
    for i in element.xpath("./parent/text()"):
        parent_inum=int(i)
        for j in element.xpath("./child/text()"):
            child_inum=int(j)
            dir_tree.addChildrenToParent(parent_inum,child_inum)
#             queue=[dir_tree]
#             while len(queue)>0:
#                 current=queue.pop(0)
#                 if parent_inum==current.name:
#                     current.addChild(child_inum)
#                 else:
#                     #array.append(current.name)
#                     queue.extend(current.children)
                    
print(dir_tree.breadthFirstSearch())
maxdepth=dir_tree.maxDepth()
output_dict["maximum depth of directory tree"]=maxdepth
print(output_dict)

<inode><id>16385</id><type>DIRECTORY</type><name/><mtime>1581231015982</mtime><permission>ec2-user:supergroup:0755</permission><nsquota>9223372036854775807</nsquota><dsquota>-1</dsquota></inode>

16385
[16385, 16386, 16387, 16390, 16412, 16401, 16391, 16388, 16413, 16402, 16392, 16393, 16394, 16395, 16396, 16397, 16398, 16399, 16400, 16389, 16414, 16415, 16416, 16417, 16418, 16419, 16420, 16421, 16422, 16403, 16404, 16405, 16406, 16407, 16408, 16409, 16410, 16411, 16385, 16386, 16387, 16390, 16412, 16401, 16391, 16388, 16413, 16402, 16392, 16393, 16394, 16395, 16396, 16397, 16398, 16399, 16400, 16389, 16414, 16415, 16416, 16417, 16418, 16419, 16420, 16421, 16422, 16403, 16404, 16405, 16406, 16407, 16408, 16409, 16410, 16411, 16385, 16386, 16387, 16390, 16412, 16401, 16391, 16388, 16413, 16402, 16392, 16393, 16394, 16395, 16396, 16397, 16398, 16399, 16400, 16389, 16414, 16415, 16416, 16417, 16418, 16419, 16420, 16421, 16422, 16403, 16404, 16405, 16406, 16407, 16408, 16409, 16410, 16411]

In [56]:
#find file size
f_size_l=[]
cnt=1
for element in tree.xpath("/fsimage/INodeSection/inode[type='FILE']/blocks"):
    #printf(element)
    blk_size = 0
    for num in element.xpath("./block/numBytes/text()"):
        blk_size +=int(num)
    f_size_l.append(blk_size)
f_size_d={}
f_size_d["max"]=max(f_size_l)
f_size_d["min"]=min(f_size_l)
output_dict["file size"]=f_size_d
print(output_dict)

{'number of files': 29, 'number of directories': 9, 'maximum depth of directory tree': 6, 'file size': {'max': 11392, 'min': 19}}


In [23]:
json.dump(output_dict,open('output.json','w'),indent=4)